# Фреймворк PyTorch для разработки искусственных нейронных сетей

## Урок 5. Face Detection and Emotion Recognition (Курсовая работа)

Нужно написать приложение, которое будет считывать и выводить кадры с веб-камеры. В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре. После этого, человек показывает жесты руками, а алгоритм должен считать их и определенным образом реагировать на эти жесты.
На то, как система будет реагировать на определенные жесты - выбор за вами. Например, на определенный жест (жест пис), система будет здороваться с человеком. На другой, будет делать скриншот экрана. И т.д.
Для распознавания жестов, вам надо будет скачать датасет https://www.kaggle.com/gti-upm/leapgestrecog, разработать модель для обучения и обучить эту модель.

*(Усложненное задание) Все тоже самое, но воспользоваться этим датасетом:
https://fitnessallyapp.com/datasets/jester/v1

# Часть 2. Использование модели

Модель обученная на датасете leapgestrecog получилась плохой. Пробовал разные модели и аугментацию, но всё-равно ничего хорошего не выходило. Думаю, это из-за того, что жесты, на которых проходит обучение, далеки от реальности. Поэтому решил сделать обучение модели на основе cv2 в режиме онлайн. Получилось гораздо лучше. Ссылка на ноутбук с моделью: https://github.com/SemenovR/GB_PyTorch/blob/lesson_5/lesson_5/lesson_5_run_train.ipynb

In [26]:
import os
import cv2
import torch
import math
import torch.nn as nn
import torchvision.transforms as tt
import matplotlib.pyplot as plt

from torch.autograd import Variable
from PIL import Image

In [27]:
os.environ['KMP_DUPLICATE_LIB_OK']='True' # Для обхода бага библиотек

In [28]:
class ResNet(nn.Module):
    def __init__(self, in_channels, num_classes, size_n, size_p, debug):
        super().__init__()
        
        self.debug = debug
        
        size_pre = in_channels
        size_cur = size_n
        size_c = size_p / 2
        self.conv1 = self.conv_block(size_pre, size_cur)
        self.conv2 = self.conv_block(size_cur, size_cur, pool=True)
        self.drop11 = nn.Dropout(0.25)
        self.res1 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop1 = nn.Dropout(0.5)
        
        size_pre = size_cur
        size_cur = size_cur * 2
        size_c = size_c / 2
        self.conv3 = self.conv_block(size_pre, size_cur)
        self.conv4 = self.conv_block(size_cur, size_cur, pool=True)
        self.drop12 = nn.Dropout(0.25)
        self.res2 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop2 = nn.Dropout(0.5)
        
        size_pre = size_cur
        size_cur = size_cur * 2
        size_c = size_c / 2
        self.conv5 = self.conv_block(size_pre, size_cur)
        self.conv6 = self.conv_block(size_cur, size_cur, pool=True)
        self.drop13 = nn.Dropout(0.25)
        self.res3 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        self.drop3 = nn.Dropout(0.5)

        #size_pre = size_cur
        #size_cur = size_cur * 2
        #size_c = size_c / 2
        #self.conv7 = self.conv_block(size_pre, size_cur)
        #self.conv8 = self.conv_block(size_cur, size_cur, pool=True)
        #self.drop14 = nn.Dropout(0.25)
        #self.res4 = nn.Sequential(self.conv_block(size_cur, size_cur), self.conv_block(size_cur, size_cur))
        #self.drop4 = nn.Dropout(0.5)
        
        size_k = 6
        size_f = int((math.floor((size_c - size_k) / size_k) + 1)**2 * size_cur)
        self.maxp = nn.MaxPool2d(size_k)
        self.flat = nn.Flatten()
        self.line = nn.Linear(size_f, num_classes)
        
    @staticmethod
    def conv_block(in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                  nn.BatchNorm2d(out_channels), 
                  nn.ELU(inplace=True)]
        if pool: layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
    

    def call_layer(self, data, layer):
        if self.debug == True:
            print(self.counter, data.shape)
            self.counter += 1
        return layer(data)
            
        
    def forward(self, xb):
        self.counter = 0
        
        out = self.call_layer(xb, self.conv1)
        out = self.call_layer(out, self.conv2)
        out = self.call_layer(out, self.drop11)
        out = self.call_layer(out, self.res1) + out
        out = self.call_layer(out, self.drop1)
        
        out = self.call_layer(out, self.conv3)
        out = self.call_layer(out, self.conv4)
        out = self.call_layer(out, self.drop12)
        out = self.call_layer(out, self.res2) + out
        out = self.call_layer(out, self.drop2)
        
        out = self.call_layer(out, self.conv5)
        out = self.call_layer(out, self.conv6)
        out = self.call_layer(out, self.drop13)
        out = self.call_layer(out, self.res3) + out
        out = self.call_layer(out, self.drop3)

        #out = self.call_layer(out, self.conv7)
        #out = self.call_layer(out, self.conv8)
        #out = self.call_layer(out, self.drop14)
        #out = self.call_layer(out, self.res4) + out
        #out = self.call_layer(out, self.drop4)
        
        out = self.call_layer(out, self.maxp)
        out = self.call_layer(out, self.flat)
        out = self.call_layer(out, self.line)
        
        return out

In [29]:
# Класс детектирования и обработки жестов с веб-камеры 
class GestureDetector(object):

    def __init__(self):
        self.channels = 1
        self.classes = 10
        self.size_n = 256
        self.size_p = 128  
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = ResNet(self.channels, self.classes, self.size_n, self.size_p, False).to(self.device)
        
        self.model.load_state_dict(torch.load('./models/model1.pth'))
        self.model.eval()

    # Функция рисования найденных параметров на кадре
    def _draw(self, frame, text):
        try:
            # пишем на кадре какой жест распознан
            cv2.putText(frame, text, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        except:
            print('Something wrong im draw function!')

        return frame
    
    @staticmethod
    def digit_to_classname(digit):
        if digit == 0:
            return 'palm'
        elif digit == 1:
            return 'I'
        elif digit == 2:
            return 'fist'
        elif digit == 3:
            return 'fist_moved'
        elif digit == 4:
            return 'thumb'
        elif digit == 5:
            return 'index'
        elif digit == 6:
            return 'ok'
        elif digit == 7:
            return 'palm_moved'
        elif digit == 8:
            return 'c'
        elif digit == 9:
            return 'down'

    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):              
        transforms = tt.Compose([# Настройки для расширения датасета
                     tt.Grayscale(num_output_channels=1), # Картинки чернобелые
                     tt.Resize((self.size_p, self.size_p), interpolation=Image.NEAREST),
                     tt.ToTensor(),
                     tt.Normalize((0.1307,), (0.3081,)),
        ])                      # Приведение к тензору

        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
            ret, frame = cap.read()
            #try:

            data = Variable(transforms(Image.fromarray(frame))).unsqueeze(0)
            data = data.to(self.device)
            
            predict = self.model(data)
            predict_text = self.digit_to_classname(predict.argmax())
            self._draw(frame, predict_text)

            #except:
            #    print('Something wrong im main cycle!')

            # Показываем кадр в окне, и называем его(окно) - 'Gesture Detection'
            cv2.imshow('Gesture Detection', frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()
        
        
# Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
cap = cv2.VideoCapture(0)  
# Создаем объект нашего класса приложения
fcd = GestureDetector()
# Запускаем
fcd.run()